ejercicio numero 5 segundo parcial


In [331]:
import os
import numpy as np
import random

from matplotlib import pyplot

from scipy import optimize
from scipy.io import loadmat

%matplotlib

Using matplotlib backend: agg


In [332]:
%mkdir data

mkdir: cannot create directory ‘data’: File exists


llamamos el archivo csv 
valores de y col 0
  1 = con cirugia
  2 = sin cirugia
col 1
  


In [333]:
genkeys = dict(missing_values={1:'nan',3:'nan',4:'nan',5:'nan',6:'nan',7:'nan',8:'nan',9:'nan',10:'nan',11:'nan',12:'nan',
                               13:"nan",14:"nan",15:"nan",16:"nan",17:"nan",18:"nan",19:"nan",20:"nan",
                               21:"nan",22:"nan",23:"nan",27:"nan",
                               },
               filling_values={1:random.choice([1,9]),3:random.uniform(36,39),4:random.randint(20, 45),5:random.randint(8, 30),6:random.randint(1, 4),7:random.randint(1, 4),8:random.randint(1, 6),9:random.randint(1,2),10:random.randint(1,5),11:random.randint(1, 4),12:random.randint(1, 4), 13:random.randint(1, 3),
                               14:random.randint(1, 3),15:random.randint(5, 8),16:random.randint(1, 4),17:random.randint(1, 5),18:random.randint(30, 50),
                               19:random.choice([6,7,5]),20:random.randint(1, 3),
                               21:random.randint(1, 3),22:random.randint(1, 3),23:random.randint(1, 2),27:random.randint(1, 2)
                               })
data = np.genfromtxt("/content/data/horse-colic.csv", delimiter=",", **genkeys)
dataP = np.genfromtxt("/content/data/horse-colic.test", delimiter=" ", **genkeys)
#data[data == '?']=0
#print(data)
x = np.delete(data,22, axis=1)
y = data[:,22]
y = np.array([int(e) for e in y])
y = np.squeeze(y)

xP = np.delete(dataP,22, axis=1)
yP = dataP[:,22]

y[y == 1] = 0
y[y == 2] = 1
y[y == 3] = 2
print(x.shape)
print(y.shape)
print(xP.shape)
print(yP.shape)
print(x)
print(y)  

(300, 27)
(300,)
(68, 27)
(68,)
[[2.00000e+00 1.00000e+00 5.30101e+05 ... 0.00000e+00 0.00000e+00
  2.00000e+00]
 [1.00000e+00 1.00000e+00 5.34817e+05 ... 0.00000e+00 0.00000e+00
  2.00000e+00]
 [2.00000e+00 1.00000e+00 5.30334e+05 ... 0.00000e+00 0.00000e+00
  1.00000e+00]
 ...
 [1.00000e+00 1.00000e+00 5.29386e+05 ... 0.00000e+00 0.00000e+00
  2.00000e+00]
 [1.00000e+00 1.00000e+00 5.30612e+05 ... 0.00000e+00 0.00000e+00
  1.00000e+00]
 [1.00000e+00 1.00000e+00 5.34618e+05 ... 0.00000e+00 0.00000e+00
  2.00000e+00]]
[1 2 0 1 1 0 0 1 2 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 2 0 0 0 2 2
 0 0 1 1 0 0 2 2 1 0 0 1 1 0 1 0 0 2 1 0 1 1 1 0 0 1 2 0 0 0 0 0 0 2 0 1 1
 1 2 0 0 1 0 1 0 0 1 0 0 1 2 0 0 0 1 0 1 0 0 0 2 0 0 0 2 0 0 1 0 0 0 2 1 1
 0 0 1 0 0 0 2 0 0 0 2 0 1 2 0 0 0 2 0 1 1 1 0 1 1 1 0 0 0 0 1 1 0 2 0 0 1
 0 0 0 1 0 0 0 0 0 2 0 0 0 1 0 2 0 2 0 0 0 2 0 1 0 1 1 0 0 1 0 2 1 0 1 0 0
 0 0 0 0 0 0 0 0 1 0 0 2 1 0 0 0 1 2 0 0 2 0 1 2 0 1 1 0 0 0 0 0 0 2 0 2 0
 0 0 0 0 1 0 2 0 0 1 0 0 0

In [334]:
#configurando parametros necesarios
input_layer_size = 27 #27 caracteristicas detalladas 
hidden_layer_size = 10 #5 unidades ocultas 
num_labels = 3        #3 etiquetas 0 , 1 y 2

#pesos theta1 y theta2
pesos = {}
pesos['Theta1'] = np.random.rand(10, 28)
pesos['Theta2'] = np.random.rand(3,11)

Theta1, Theta2 = pesos['Theta1'], pesos['Theta2']
print(Theta1.ravel().shape)
print(Theta2.ravel().shape)

nm_params = np.concatenate([Theta1.ravel(),Theta2.ravel()])
print(nm_params.shape)

(280,)
(33,)
(313,)


In [335]:
#funcion de la sigmoide
#sigmoide sirve para calcular los valores del forward
def sigmoid(z):
  return 1.0/(1.0+np.exp(-z))

#sirve para el back propagtion
def sigmoidGradient(z):
  g = np.zeros(z.shape)
  g = sigmoid(z)*(1-sigmoid(z))
  return g

In [336]:
# calculamos el costo entre lo que predice y lo que tiene que predecir ---- regrecion logistica
def nnCostFunction(nm_params, input_layer_size, hidden_layer_size, num_labels, x, y, lambda_=0.0):
  Theta1 = np.reshape(nm_params[:hidden_layer_size * (input_layer_size + 1)],
                      (hidden_layer_size, (input_layer_size + 1)))
  Theta2 = np.reshape(nm_params[(hidden_layer_size * (input_layer_size + 1)):],
                      (num_labels, (hidden_layer_size +1)))
  m = y.size

  J = 0
  #definimos theta 1 y 2 con los tamaños de de theta 1 y2
  Theta1_grad = np.zeros(Theta1.shape)
  Theta2_grad = np.zeros(Theta2.shape)
  #gradiente de propagacion hacia adelante
  #calculo de forward se concatena fila de unos 
  a1 = np.concatenate([np.ones((m, 1)),x], axis = 1)

  a2 = sigmoid(a1.dot(Theta1.T))
  a2 = np.concatenate([np.ones((a2.shape[0],1)),a2], axis = 1)

  a3 = sigmoid(a2.dot(Theta2.T))

  y_matrix = y.reshape(-1)

  y_matrix = np.eye(num_labels)[y_matrix]

  temp1 = Theta1
  temp2 = Theta2

  #termino de regularizacion
  reg_term = (lambda_ /(2*m))*(np.sum(np.square(temp1[:,1:]))+np.sum(np.square(temp2[:,1:])))

  J = (-1 / m) * np.sum((np.log(a3) * y_matrix) + np.log(1 - a3) * (1 - y_matrix)) + reg_term

  #back propagation
  delta_3 = a3 - y_matrix
  delta_2 = delta_3.dot(Theta2)[:, 1:] * sigmoidGradient(a1.dot(Theta1.T))

  Delta1 = delta_2.T.dot(a1)
  Delta2 = delta_3.T.dot(a2)

  # Agregar regularización al gradiente
  Theta1_grad = (1 / m) * Delta1
  Theta1_grad[:, 1:] = Theta1_grad[:, 1:] + (lambda_ / m) * Theta1[:, 1:]
    
  Theta2_grad = (1 / m) * Delta2
  Theta2_grad[:, 1:] = Theta2_grad[:, 1:] + (lambda_ / m) * Theta2[:, 1:]

  grad = np.concatenate([Theta1_grad.ravel(), Theta2_grad.ravel()])
    
  return J, grad


In [337]:
lambda_ = 0
J, _ = nnCostFunction(nm_params, input_layer_size, hidden_layer_size, num_labels, x, y, lambda_)
print('Costo en parametros (cargado de  ): %.6f ' % J)


Costo en parametros (cargado de  ): 10.233546 


In [338]:
def randInitializeWeights(L_in, L_out, epsilon_init=0.12):
    W = np.zeros((L_out, 1 + L_in))
    W = np.random.rand(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init

    return W

In [339]:
print('Inicialización de parámetros de redes neuronales...')

initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size)
initial_Theta2 = randInitializeWeights(hidden_layer_size, num_labels)

# Desenrrollr parametros
initial_nn_params = np.concatenate([initial_Theta1.ravel(), initial_Theta2.ravel()], axis=0)

Inicialización de parámetros de redes neuronales...


In [340]:
options= {'maxiter': 10000}

#  You should also try different values of lambda
lambda_ = 1

# Create "short hand" for the cost function to be minimized
costFunction = lambda p: nnCostFunction(p, input_layer_size,
                                        hidden_layer_size,
                                        num_labels, x, y, lambda_)

# Now, costFunction is a function that takes in only one argument
# (the neural network parameters)
res = optimize.minimize(costFunction, initial_nn_params, jac=True, method='TNC', options=options)

# get the solution of the optimization
nm_params = res.x
        
# Obtain Theta1 and Theta2 back from nn_params
Theta1 = np.reshape(nm_params[:hidden_layer_size * (input_layer_size + 1)],
                    (hidden_layer_size, (input_layer_size + 1)))

Theta2 = np.reshape(nm_params[(hidden_layer_size * (input_layer_size + 1)):],
                    (num_labels, (hidden_layer_size + 1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.


In [341]:
def predict(Theta1, Theta2, x):
    """
    Predict the label of an input given a trained neural network
    Outputs the predicted label of X given the trained weights of a neural
    network(Theta1, Theta2)
    """
    # Useful values
    m = x.shape[0]
    num_labels = Theta2.shape[0]

    # You need to return the following variables correctly
    p = np.zeros(m)
    h1 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), x], axis=1), Theta1.T))
    h2 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), h1], axis=1), Theta2.T))
    p = np.argmax(h2, axis=1)
    return p

In [342]:
pred = predict(Theta1, Theta2, x[:,:])
print(pred)
print('Training Set Accuracy: %f' % (np.mean(pred == y[:]) * 100))

[1 0 0 0 1 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0
 0 0 0 1 0 0 0 0 1 1 0 1 1 0 0 0 0 1 0 0 1 1 1 0 0 1 0 0 0 0 0 1 0 1 0 0 1
 1 0 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1
 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 1 1 0 1 0 1 0 0 0 0 1 1 0 0 0 0 1
 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 1 0 0 1 1 1 0 0 1 1 0 0 0 0 0 1 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0
 0 1 0 0]
Training Set Accuracy: 76.333333


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.


In [343]:
pred = predict(Theta1, Theta2, xP[:,:])
print(pred)
print('Training Set Accuracy: %f' % (np.mean(pred == yP[:]) * 100))

[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0
 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
Training Set Accuracy: 10.294118


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.
